<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/kaggle_bi_lstm_crf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import keras
print(keras.__version__)

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from math import nan
from keras.callbacks import ModelCheckpoint

!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.layers import CRF

from google.colab import drive
drive.mount('/content/drive/')

2.4.3
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-3sxgd_ut
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-3sxgd_ut
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=4e78960208131ec2b4eb42dc53fb91c866cda1944900cc678bf37ef72b43968f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d1t_onxx/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Mounted at /content/drive/


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ner.csv', encoding="ISO-8859-1", error_bad_lines=False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [4]:
df

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050790,1048570,they,respond,to,TO,lowercase,to,VBD,lowercase,responded,PRP,O,said,VBD,O,forc,NNS,lowercase,forces,lowercase,said,47959.0,lowercase,they,O
1050791,1048571,respond,to,the,DT,lowercase,the,TO,lowercase,to,VBD,O,they,PRP,O,said,VBD,lowercase,said,lowercase,they,47959.0,lowercase,responded,O
1050792,1048572,to,the,attack,NN,lowercase,attack,DT,lowercase,the,TO,O,respond,VBD,O,they,PRP,lowercase,they,lowercase,responded,47959.0,lowercase,to,O
1050793,1048573,the,attack,with,IN,lowercase,with,NN,lowercase,attack,DT,O,to,TO,O,respond,VBD,lowercase,responded,lowercase,to,47959.0,lowercase,the,O


In [6]:
df.columns

Index(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'pos', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word', 'sentence_idx', 'shape',
       'word', 'tag'],
      dtype='object')

In [7]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
getter = SentenceGetter(df)

In [10]:
sentences = getter.sentences

In [11]:
print(sentences[5])

[('The', 'O'), ('party', 'O'), ('is', 'O'), ('divided', 'O'), ('over', 'O'), ('Britain', 'B-gpe'), ("'s", 'O'), ('participation', 'O'), ('in', 'O'), ('the', 'O'), ('Iraq', 'B-geo'), ('conflict', 'O'), ('and', 'O'), ('the', 'O'), ('continued', 'O'), ('deployment', 'O'), ('of', 'O'), ('8,500', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('in', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O'), ('The', 'O'), ('party', 'O'), ('is', 'O'), ('divided', 'O'), ('over', 'O'), ('Britain', 'B-gpe'), ("'s", 'O'), ('participation', 'O'), ('in', 'O'), ('the', 'O'), ('Iraq', 'B-geo'), ('conflict', 'O'), ('and', 'O'), ('the', 'O'), ('continued', 'O'), ('deployment', 'O'), ('of', 'O'), ('8,500', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('in', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O')]


In [13]:
# 문장 최대 길이

maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 140


In [16]:
words = list(set(df["word"].values))

In [18]:
words.append("ENDPAD")

In [20]:
n_words = len(words); n_words

30174

In [22]:
#태그 할당
#unk는 
tags = []
for tag in set(df["tag"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
print(tags)

['B-org', 'I-gpe', 'unk', 'B-per', 'B-tim', 'B-eve', 'B-gpe', 'I-eve', 'B-art', 'I-per', 'B-geo', 'I-org', 'I-art', 'I-geo', 'I-tim', 'O', 'I-nat', 'B-nat']


In [23]:
n_tags = len(tags); n_tags

18

In [24]:
#단어 -> 숫자로 바꿔줌
from future.utils import iteritems
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [25]:
tag2idx

{'B-art': 8,
 'B-eve': 5,
 'B-geo': 10,
 'B-gpe': 6,
 'B-nat': 17,
 'B-org': 0,
 'B-per': 3,
 'B-tim': 4,
 'I-art': 12,
 'I-eve': 7,
 'I-geo': 13,
 'I-gpe': 1,
 'I-nat': 16,
 'I-org': 11,
 'I-per': 9,
 'I-tim': 14,
 'O': 15,
 'unk': 2}

In [26]:
idx2tag

{0: 'B-org',
 1: 'I-gpe',
 2: 'unk',
 3: 'B-per',
 4: 'B-tim',
 5: 'B-eve',
 6: 'B-gpe',
 7: 'I-eve',
 8: 'B-art',
 9: 'I-per',
 10: 'B-geo',
 11: 'I-org',
 12: 'I-art',
 13: 'I-geo',
 14: 'I-tim',
 15: 'O',
 16: 'I-nat',
 17: 'B-nat'}

In [28]:
#일정한 길이로 맞춰줌
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [29]:
np.array(X).shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(35177,)

In [30]:
X = pad_sequences(maxlen=140, sequences=X, padding="post",value=n_words - 1)

In [31]:
y_idx = [[tag2idx[w[1]] for w in s] for s in sentences]
print(sentences[100])
print(y_idx[100])

[('The', 'O'), ('Pakistani', 'B-gpe'), ('military', 'O'), ('launched', 'O'), ('its', 'O'), ('offensive', 'O'), ('in', 'O'), ('Orakzai', 'B-geo'), ('to', 'O'), ('hunt', 'O'), ('Taliban', 'B-org'), ('insurgents', 'O'), ('.', 'O'), ('The', 'O'), ('Pakistani', 'B-gpe'), ('military', 'O'), ('launched', 'O'), ('its', 'O'), ('offensive', 'O'), ('in', 'O'), ('Orakzai', 'B-geo'), ('to', 'O'), ('hunt', 'O'), ('Taliban', 'B-org'), ('insurgents', 'O'), ('.', 'O')]
[15, 6, 15, 15, 15, 15, 15, 10, 15, 15, 0, 15, 15, 15, 6, 15, 15, 15, 15, 15, 10, 15, 15, 0, 15, 15]


In [32]:
y = pad_sequences(maxlen=140, sequences=y_idx, padding="post", value=tag2idx["O"])
print(y_idx[100])

[15, 6, 15, 15, 15, 15, 15, 10, 15, 15, 0, 15, 15, 15, 6, 15, 15, 15, 15, 15, 10, 15, 15, 0, 15, 15]


In [33]:
#원핫인코딩
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [34]:
#데이터셋 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [54]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as keras
from keras.models import Sequential
from keras.models import load_model
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

In [67]:
input = Input(shape=(140,))
word_embedding_size = 300
model = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=140)(input)
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.5, 
                           recurrent_dropout=0.5, 
                           kernel_initializer=keras.initializers.he_normal()))(model)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer=keras.initializers.he_normal())(model)
model = TimeDistributed(Dense(n_tags, activation="relu"))(model)  # previously softmax output layer

crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [69]:
model = Model(input, out)

In [70]:
adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
#model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=adam, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [71]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 140)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 140, 300)          9052200   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 140, 600)          1442400   
_________________________________________________________________
lstm_13 (LSTM)               (None, 140, 600)          2882400   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 140, 18)           10818     
_________________________________________________________________
crf_4 (CRF)                  (None, 140, 18)           702       
Total params: 13,388,520
Trainable params: 13,388,520
Non-trainable params: 0
_______________________________________________

In [72]:
# Saving the best only
filepath="ner-bi-lstm-td-model-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [75]:
history = model.fit(X_train, 
                    np.array(y_train), 
                    batch_size=256, 
                    epochs=20, 
                    validation_split=0.2)

Epoch 1/20


AttributeError: ignored